In [1]:
# Local
from langchain_community.chat_models import ChatOllama

llama2_chat = ChatOllama(model="llama2:13b-chat")
llama2_code = ChatOllama(model="codellama:7b-instruct")

# Simply set the LLM we want to use
llm = llama2_chat

### Loading all doc in dir

In [5]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader('test dir', glob="**/*.md")
docs = loader.load()
len(docs)

1